In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

94773248/94765736 [==============================] - 1s 0us/step


In [2]:
# deconv module
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

In [3]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

In [5]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')
upconv = _make_deconv_layer(3)
final_layer = tf.keras.layers.Conv2D(1, kernel_size=(1,1), padding='same')

# input :  192x256
# output : 48x64
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

# np_input = np.zeros((1,256,192,3), dtype=np.float32)
np_input = np.random.randn(1,256,192,3)
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=np.float32)
print (tf_input.shape) # TensorShape([1,256,192,3])

tf_output = model(tf_input)

print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 192, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
sequential_1 (Sequential)    (None, 64, 48, 256)       10489600  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 48, 1)         257       
Total params: 34,077,569
Trainable params: 34,022,913
Non-trainable params: 54,656
_________________________________________________________________
(1, 256, 192, 3)
(1, 64, 48, 1)
tf.Tensor(
[[[ 0.0005819 ]
  [-0.00031189]
  [-0.02409783]
  [-0.01189323]
  [-0.01007731]
  [ 0.01841793]
  [-0.03823639]
  [-0.00464023]
  [-0.00489871]
  [ 0.00405288]]

 [[ 0.00663753]
  [-0.02913981]
  [-0

In [ ]:

human keypoint detection
HP

Top-down
모든 사람의 정확한 keypoint 를 찾기 위해 object detection 을 사용
crop 한 이미지 내에서 keypoint 를 찾아내는 방법으로 표현
detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야하기 때문에 사람이 많이 등장할 때는 느리다는 단점이 있다.
Bottom-up
detector가 없고 keypoint 를 먼저 검출 예로 손목에 해당하는 모든 점들을 검출
한 사람에 해당하는 keypoint 를 clustering
detector 가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않다.
top down 방식에 비해 keypoint 검출범위가 넓어 성능이 떨어진다는 단점이 있다.
Convolutional Pose Machines
CVPR 2016에서 발표된 CPM 은 completely differentiable 한 multi-stage 구조를 제안했습니다. multi stage 방법들은 DeepPose 에서부터 지속적으로 사용되어 왔었습니다. 하지만 crop 연산 등 비연속적인 미분불가능한 stage 단위로 나눠져 있었기 때문에 학습 과정을 여러번 반복하는 비효율적인 방법을 사용해왔습니다.

Convolutional Pose Machines
CPM 은 end-to-end 로 학습할 수 있는 모델을 제안합니다.1
Stage 1 은 image feature 를 계산하는 역할을 하고 stage 2는 keypoint 를 예측하는 역할을 합니다. g1과 g2 모두 heatmap 을 출력하게 만들어서 재사용이 가능한 부분은 weight sharing 할 수 있도록 세부 모델을 설계 했습니다.

2

Stage ≥ 2 에서 볼 수 있듯이 stage 2 이상부터는 반복적으로 사용할 수 있습니다. 보통은 3개의 스테이지를 사용한다고 합니다. stage 1 구조는 고정이고 stage 2 부터는 stage 2 구조를 반복해서 추론합니다. stage 2 부터는 입력이 heatmap(image feature)이 되기 때문에 stage 단계를 거칠수록 keypoint가 refinement 되는 효과를 볼 수 있습니다.3
사실 CPM 이 아주 좋은 방법이라고는 말하기 어렵습니다. Multi-stage 방법을 사용하기 때문에 end-to-end 로 학습이 가능하더라도 그대로 학습하는 경우는 높은 성능을 달성하기 어렵습니다. 따라서 stage 단위로 pretraining 을 한 후 다시 하나의 모델로 합쳐서 학습을 합니다. 논문을 작성하기 위해서라면 충분히 감내할 수 있지만 서비스 측면에서 바라본다면 불편한 요소라고 할 수 있습니다. 이런 문제점들은 후에 제안되는 모델들이 적극적으로 개선하고 있습니다.

CPM 을 다루는 이유는 성능 때문입니다. receptive field 를 넓게 만드는 multi stage refinement 방법이 성능향상에 크게 기여한 것 같습니다.

4주황색 실선이 Tompson 알고리즘입니다. CPM 에서 제안한 검정색, 회색 실선이 detection rate에서 유의미한 차이를 보이고 있는 것을 볼 수 있습니다. MPII 의 PCKh@0.5 에서 87.95% 를 달성했다고 합니다. 당시 2등보다 6.11%p 높은 성능을 보였습니다.

Stacked Hourglass Network
Stacked Hourglass Networks for Human Pose Estimation
Stacked Hourglass Network 의 기본 구조는 모래시계 같은 모양으로 만들어져 있습니다. Conv layer 와 pooling 으로 이미지(또는 feature) 를 인코딩 하고 upsampling layer 를 통해 feature map 의 크기를 키우는 방향으로 decoding 합니다. feature map 크기가 작아졌다 커지는 구조여서 hourglass 라고 표현합니다.

shn

기존 방법들과의 가장 큰 차이점은

feature map upsampling residual connection 이라고 할 수 있을 것 같습니다.

pooling으로 image의 global feature를 찾고 upsampling으로 local feature를 고려하는 아이디어가 hourglass의 핵심 novelty라고 할 수 있습니다.

비교
비교
(a) : Hourglass (b) : CPN(cascaded pyramid networks) (c) : SimpleBaseline - transposed conv (d) : SimpleBaseline - dilated conv

Simplebaseline 이 다른 알고리즘들에 비해 성능이 떨어지지 않지만 구조를 보면 공통점과 차이점이 있는데 무엇일까
공통점 : high resolution → low resolution 인 encoder 와 low → high 인 decoder 구조로 이루어진 점

차이점 : Hourglass 는 encoder 와 decoder 의 비율이 거의 비슷함(대칭적임). 반면 Simplebaseline 은 encoder 가 무겁고 (resnet50 등 backbone 사용) decoder 는 가벼운 모델을 사용함. (a), (b) 는 skip connection 이 있지만 (c) 는 skip connection 이 없다.

기존 모델들은 skip connection 을 적극적으로 사용했는데 왜 사용했을까
pooling(strided conv) 할 때 소실되는 정보를 high level layer에서 사용해서 detail한 정보를 학습하기 위해 사용합니다. 당연히 사용할 때 성능이 더 좋을 것 같습니다.

HR-NET
hrnet

HRNet 또한 이전 알고리즘 들과 마찬가지로 heatmap을 regression하는 방식으로 학습하고 MSE loss를 이용합니다. (특히 Simplebaseline 과 거의 유사합니다.)
HR-NET github
SimpleBaseline 코드구조
sb

encoder : conv layers
decoder : deconv module + upsampling
Simple Baselines for Human Pose Estimation and Tracking
conv model로 resnet을 사용
deconv module은 deconv-bn-relu 이 단계가 3개의 레이어로 이루어져있고, deconv는 256 filter size, 4x4 kernel, stride 2 로 2배씩 feature map이 커집니다.
마지막 출력 레이어는 k 개의 1x1 conv layer로 구성
microsoft/human-pose-estimation.pytorch

nn. 표현이 많이 등장합니다. torch.nn 으로 keras.layers 와 같이 딥러닝 모델 구성에 필요한 도구들이 정의되어 있습니다.

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L29

29번째 줄에서는 BasicBlock 이라는 클래스가 보이네요. keras.models 로 model 을 선언하는 것과 비슷합니다.

참고로 pytorch model 에서는 사용된 layer 를 forward 함수를 통해 computational graph 를 그려줍니다.

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L42

forward 함수를 읽어볼까요? 앗.. 어딘가 많이 본 구조 아닌가요?

    residual = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)

    if self.downsample is not None:
        residual = self.downsample(x)

    out += residual
    out = self.relu(out)
맞습니다. residual block 을 사용했네요.

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L157

Pose 메인 model 을 살펴보니 4개의 residual block 을 이용합니다. 완전 resnet 과 동일하죠?

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L234

forward 함수를 보면 흐름을 쉽게 알 수 있습니다.

def forward(self, x): x = self.conv1(x) x = self.bn1(x) x = self.relu(x) x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.deconv_layers(x)
    x = self.final_layer(x)

    return x
resnet 을 통과한 후 deconv_layers 와 final_layer를 차례로 통과합니다.

deconv layer 를 찾아보니,

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L219

    layers.append(
            nn.ConvTranspose2d(
                in_channels=self.inplanes,
                out_channels=planes,
                kernel_size=kernel,
                stride=2,
                padding=padding,
                output_padding=output_padding,
                bias=self.deconv_with_bias))
        layers.append(nn.BatchNorm2d(planes, momentum=BN_MOMENTUM))
        layers.append(nn.ReLU(inplace=True))
transpose conv 와 bn, relu 로 이루어져있는 것을 확인했습니다.

세세한 파라미터는 어디에 있을까요?

EXTRA 가 자주 등장하는 것을 볼 때, 어떤 configuration 파일이 있을 것으로 짐작해 볼 수 있겠네요. repo 내에서 검색해보면 파라미터 관련 정보를 담고 있는 아래 파일을 찾을 수 있습니다.

https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/experiments/coco/resnet50/256x192d256x3adam_lr1e-3.yaml#L23

NUM_DECONV_LAYERS: 3 NUM_DECONV_FILTERS:

- 256
- 256
- 256
NUM_DECONV_KERNELS:
- 4
- 4
- 4
deconv layer 의 파라미터가 아주 상세히 적혀있네요!